<a href="https://colab.research.google.com/github/JillHero/Colab/blob/main/whatsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 所有关于Facebook账号的操作都不要在中国ip下使用,Facebook开发者账号不要使用VPN请求。否则会被遣返或封号

whatsApp是Meta公司旗下的一款跨平台加密即时通讯应用程序，可以使用互联网和网络电话号码进行视频、语音通讯，向其他用户发送短信、文档文件、PDF文件、图片、影片、音乐、联系人信息、用户位置及录音档等。
[截止2020年活跃用户超过1亿的社交平台](https://en.wikipedia.org/wiki/List_of_social_platforms_with_at_least_100_million_active_users)
# 一、建立Facebook开发者账号
# 二、添加WhatsApp产品并配置
# 三、发送测试消息
# 四、webhook
# 五、deepLinking

In [ ]:
"https://61962de0.whatsapp-login-sequence.pages.dev/"

# 一、建立Facebook开发者账号
## 准备工作
1. 一台可以使用浏览器带有GUI的海外服务器(ubuntu、ws等),因为Facebook地区校验十分严格,单单使用vpn会被检测出来导致封号。
2. 两个可以收取验证码的海外手机号
3. 一个邮箱号(最好是google、outlook等海外邮箱)

## 注册账号
1. 注册账号需要注册[Meta开发者](https://developers.facebook.com/)也就是Facebook账号
2. 注册[MetaBusiness](https://business.facebook.com/)，前者用来创建WhatsApp产品，后者用来管理业务资产后续正式上线时需要用到。
3. 完成两步认证
>注意:Facebook账号注册完成后大概率会被冻结，需要提交资料1天后解封。解封后先创建一个WhatsApp产品，流程在第二步，如果看不见WhatsApp产品，说明这台服务器的ip不被允许使用WhatsApp，或者这个账号无意间在中国ip登陆过，这个时候就只能重新注册账号。能看见WhatsApp产品后再注册MetaBusiness账号，这时也会被冻结1天，需要立即提交手机号申诉。申诉也需要等待一天，如果申诉成功这个账号就可以使用，如果申诉不成功就需要重头开始注册。注册完成后必须添加两步验证，因为在最后上线时meta要求开发者的business账号拥有两步验证才可以绑定正式的whatsApp business手机号。但是可能添加两步验证时meta又认为使用的是新的设备，需要等待几天才可以添加。

# 二、添加WhatsApp产品并配置
## 添加WhatsApp产品
使用此链接[添加应用](https://developers.facebook.com/apps/?show_reminder=true)创建一个应用选择 其他->商务->添加应用名称并创建->添加WhatsApp和Webhook应用
## 配置
1. 在侧边栏找到WhatsApp进入API设置，这时Meta会自动创建一个测试电话号码。
2. 测试电话号码只能用作沙盒环境，需要用到一个真实的电话号码来充当正式环境的手机号
  1. 创建正式环境的手机号需要满足一下条件
    1. facebook账号需要开启两步认证，最好是googleAuthenticator
    2. 需要公司信息包括公司名、网站、地址
3. 下载WhatsApp app并创建一个真实的WhatsApp账号，需要用到一个海外手机号。和上面的正式环境手机号不同，一个是发送方，一个是接受方
4. 创建模版
  1. [创建模版](https://business.facebook.com/wa/manage/message-templates/)
    1. 在这里选择身份验证或者交易相关，名字会在发消息时用到，语言需要选择对应的。其中English分为: English、English(US)、English(UK)在发消息时严格区分。[![image.png](https://i.postimg.cc/LXjkVttC/image.png)](https://postimg.cc/gXk6z6B8)
    2. 添加正文消息，可选的页眉、页脚和按钮。每个都可以添加变量，在模板中以`{{1}}`表示。发送消息时以位置参数来对应[![image.png](https://i.postimg.cc/Lsx3XttG/image.png)](https://postimg.cc/kVtbYthy)

# 三、发送测试消息
创建完模板后就可以使用模板来发送测试消息

In [ ]:
!pip install requests

In [ ]:
import requests
import json
from google.colab import userdata
"""
facebook的所有api内容都是以graphQL的形式来提供，所有产品的API都是以https://graph.facebook.com/{version}/xxx开头
"""

from_id = "218901994645066"
token = userdata.get('META_TOKEN')
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
}

# https://developers.facebook.com/docs/whatsapp/cloud-api/reference/messages

data = {
    "messaging_product": "whatsapp",
    "to": "447418349435",
    "type": "template",
    "template": {
         "name": "welcome",  # 创建模板时的名字
         "language": { "code": "en" }, # 创建模板时的语言
         "components": [{ # 内容数组
            "type": "body", # 内容的位置 正文：body 页眉： header 按钮：button
            "parameters": [{ # 参数数组，按照模板{{1}}、{{2}}来传
            "type": "text", # 参数类型
            "text": "this is tokenxxxxxx" # 参数内容
          }]}]
         }
    }
requests.post('https://graph.facebook.com/v18.0/218901994645066/messages',
              headers = headers,
              data = json.dumps(data))

# 四、Webhook
WhatsApp的大部分消息通知都是以Webhook的形式通知给服务器,包括消息是否接受、用户手机号是否变更
## 配置
[webhook](https://developers.facebook.com/apps/914879833574141/whatsapp-business/wa-setting)
1. 需要有一个服务器地址用作回调地址，验证口令需要自定义，在Facebook回调的时候他会通过query传过来，我们需要进行校验来验证真假。[![image.png](https://i.postimg.cc/ZK6VQXGV/image.png)](https://postimg.cc/2V8Qb98Z)
2. 订阅webhook字段，通过订阅的字段Facebook发送不同的请求。通过不同的请求服务器处理相应的操作。Facebook建议全部订阅 [![image.png](https://i.postimg.cc/jjNZNmRp/image.png)](https://postimg.cc/VSfqP7rg)

## 测试
1. 启动服务器
2. 点击测试[![image.png](https://i.postimg.cc/ncpWYH5n/image.png)](https://postimg.cc/c6kTdN0z)


# deepLinking
[如何从不同的应用程序链接 WhatsApp
](https://faq.whatsapp.com/425247423114725/?cms_platform=iphone&helpref=platform_switcher)

whatsApp支持多种方式和其他app链接
比如通过链接的方式只需要知道商户手机号，生成链接，用户通过点击链接的方式唤起whatsApp并填充信息

